In [1]:
%matplotlib inline 
from IPython.display import display
from os.path import isfile, join, exists
from os import listdir, makedirs, walk, remove, getlogin
from nilearn import image, plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


/jukebox/pkgs/ANACONDAPY/ni/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Set subject-specific parameters

# Set subject ID
subid = 'impr_g1_s3_020518'

# manually set scan num, type, and conds
epis = [9, 10, 11, 12]
anat = [8]
fieldmaps = [13, 14]

epi_cond = ['neg1', 'pos1', 'mixed1']

In [3]:
### Set experiment-specific parameters #####

# Set data dir
projdir = '/mnt/bucket/labs/hasson/mai/projects/impressions/data'

#Set code dir
codedir = '/mnt/bucket/labs/hasson/mai/code/preproc/bash'

# Set rawdata path
rawdir = join(projdir, subid, 'raw', 'dcm')

# Set savefile
savefile = join(projdir, subid, 'scanInfo.csv')

# Set output dir
outputdir = join(projdir, subid, 'raw', 'niftis_good')
if not exists(outputdir):
    print(outputdir + ' does not exist, creating')
    makedirs(outputdir)

In [4]:
### Convert dicoms to niftis ####
# This script copies dicoms from the raw directory to the nifti preproc dir.
# Then it unzips everything and converts dicoms to niftis. Finally, it deletes
# the original dicoms.
!{join(codedir, 'prep_and_dcm2nii.sh')} {rawdir} {outputdir}

copying dicoms
unzipping dicoms
converting dicms to nifti
Chris Rorden's dcm2niiX version 12Feb2016 (64-bit)
Version 12Feb2016 (64-bit)
Found 2425 DICOM image(s)
Note: intensity range requires saving as rare 16-bit UNSIGNED integer. Subsequent tools may require 32-bit conversion
Convert 592 DICOM as /mnt/bucket/labs/hasson/mai/projects/impressions/data/impr_g1_s3_020518/raw/niftis_good/10_EPI-2mm-CMRR-PPF78-1500TR-MB4a (96x96x60x592)
compress: "/usr/bin/pigz" -n -f "/mnt/bucket/labs/hasson/mai/projects/impressions/data/impr_g1_s3_020518/raw/niftis_good/10_EPI-2mm-CMRR-PPF78-1500TR-MB4a.nii"
Note: intensity range requires saving as rare 16-bit UNSIGNED integer. Subsequent tools may require 32-bit conversion
Convert 590 DICOM as /mnt/bucket/labs/hasson/mai/projects/impressions/data/impr_g1_s3_020518/raw/niftis_good/11_EPI-2mm-CMRR-PPF78-1500TR-MB4a (96x96x60x590)
compress: "/usr/bin/pigz" -n -f "/mnt/bucket/labs/hasson/mai/projects/impressions/data/impr_g1_s3_020518/raw/niftis_good/11_EP

In [4]:
# Create subject information file

scanNum = epis + anat + fieldmaps 
scanType = ['epi', 'epi', 'epi', 'epi', 'anat', 'fieldmap', 'fieldmap']
scanCond = epi_cond + ['pieman', 'anat', 'se_fieldmap_ap', 'se_fieldmap_pa' ]

# automatically get scan names
scanNames = np.array([])
niftis_all = listdir(outputdir)
for i in scanNum:
    for filename in niftis_all:
        if str(i) + '_' in filename:
            scanNames = np.append(scanNames, filename)
            break       
            
# make dataframe
df = pd.DataFrame({'scanNum': scanNum,'scanType': scanType, 'scanName':scanNames}, index=scanCond)
df


,scanName,scanNum,scanType
neg1,09_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz,9,epi
pos1,10_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz,10,epi
mixed1,11_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz,11,epi
pieman,12_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz,12,epi
anat,08_T1w_MPR_1iso_iP2.nii.gz,8,anat
se_fieldmap_ap,13_SpinEchoFieldMap_AP.nii.gz,13,fieldmap
se_fieldmap_pa,14_SpinEchoFieldMap_PA.nii.gz,14,fieldmap


In [24]:
# if everything looks fine, save dataframe to subjects dir
df.to_csv(savefile)

print('**** done!')

**** done!
